In [126]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
import pandas as pd
from __future__ import division
np.set_printoptions(precision=3)

# Define properties

In [127]:
number_of_groups = 4
width = 154
fuel_nodes = 4
reflector_nodes = 4

In [128]:
# Cross sections ordered from lowest energy to highest energy

# fission is actually fission * neutrons per fission
fuel_fission = np.array([
        0.009572,
        0.001193,
        0.01768,
        0.18514
    ])

fuel_transport = np.array([
        1.38741155,
        2.76065151,
        4.74833808,
        8.46740051
    ])

fuel_scattering = np.array([
        [0, 0.083004, 0,      0      ],
        [0, 0,        0.0584, 0      ],
        [0, 0,        0,      0.06453],
        [0, 0,        0,      0      ]
    ])

fuel_absorption = np.array([
        0.004946,
        0.002840,
        0.03053,
        0.1210
    ])

fuel_removal = np.array([
        0.08795,
        0.06124,
        0.09506,
        0.1210
    ])

# Not a cross section. 1 / (3 sigma_tr)
fuel_diffusion = np.array([
        2.1623,
        1.0867,
        0.6318,
        0.3543
    ])

# Fraction of fission neutrons born into each group
fuel_chi = np.array([
        1.,
        0.,
        0.,
        0.
    ])

In [129]:
# Reflector properties

reflector_fission = np.array([
        0, 0, 0, 0
    ])

reflector_transport = np.array([
        0.20608,
        0.60215,
        0.56830,
        1.21110
    ])

reflector_absorption = np.array([
        0.00051,
        0.00354,
        0.01581,
        0.04637
    ])

reflector_scattering = np.array([
        [0.37045, 0.04152, 0.00001, 0.00000],
        [0.00000, 0.98285, 0.07459, 0.01371],
        [0.00000, 0.00000, 0.76110, 0.31856],
        [0.00000, 0.00000, 0.00085, 1.96607]
    ])

reflector_diffusion = 1 / (3 * reflector_transport)

reflector_chi = np.array([
        0, 0, 0, 0
    ])

In [156]:
class Material(object):
    """Material for a reactor
    """

    def __init__(self, number_of_groups):
        self._groups = number_of_groups
        self._scattering = np.zeros((number_of_groups, number_of_groups))
        self._absorption = np.zeros((number_of_groups))
        self._fission = np.zeros((number_of_groups))
        self._width = 10.
        self._transport = np.zeros((number_of_groups))
        self._neutrons_per_fission = 2.43
        self._chi = np.zeros((number_of_groups))

    @property
    def scattering(self):
        """The scattering cross sections

        This should be a (g * g) matrix where the (i,j)th element
        is the cross section for scattering from energy group i to
        energy group j.
        """
        return self._scattering

    @scattering.setter
    def scattering(self, matrix):
        # Verify that the matrix is g x g
        try:
            assert matrix.shape == (self._groups, self._groups)
        except:
            raise TypeError("Must have scattering from and to each group")
        
        self._scattering = matrix

    @property
    def fission(self):
        """The fission cross sections

        This should be a length-g array where g is the number
        of energy groups.
        """
        return self._fission

    @fission.setter
    def fission(self, fission):
        self._fission = fission

    @property
    def absorption(self):
        """Absorption cross sections

        This should be a length-g array where g is the number
        of energy groups.
        """
        return self._absorption

    @absorption.setter
    def absorption(self, absorption):
        # Verify g groups
        try:
            assert absorption.shape[0] == (self._groups)
        except:
            error_string = "Your value has shape {0}, should have {1}"
            raise TypeError(error_string.format(absorption.shape, (self._groups)))
        self._absorption = absorption

    @property
    def transport(self):
        """Transport cross sections

        This should be a length-g array where g is the number
        of energy groups.
        """
        return self._transport

    @transport.setter
    def transport(self, transport):
        try:
            assert transport.shape[0] == (self._groups)
        except AssertionError:
            error_string = "Your value has shape {0}, should have {1}"
            raise TypeError(error_string.format(transport.shape, (self._groups)))
        self._transport = transport

    @property
    def width(self):
        """Width

        The one-dimensional width of the material in the core.
        """
        return self._width

    @width.setter
    def width(self, w):
        # Width should be a float
        try:
            assert type(w) == type(7.)
        except:
            raise TypeError("Width should be a float")
        self._width = width
    
    @property
    def neutrons_per_fission(self):
        return self._neutrons_per_fission
    
    @neutrons_per_fission.setter
    def neutrons_per_fission(self, nu):
        self._neutrons_per_fission = nu
    
    @property
    def chi(self):
        return self._chi
    
    @chi.setter
    def chi(self, n_yield):
        self._chi = n_yield

In [157]:
fuel = Material(number_of_groups)
reflector = Material(number_of_groups)

In [192]:
fuel.width = 154.
fuel.transport = fuel_transport
fuel.absorption = fuel_absorption
fuel.fission = fuel_fission
fuel.scattering = fuel_scattering
fuel.chi = np.array([1., 0., 0., 0.])

reflector.width = 10.
reflector.transport = reflector_transport
reflector.absorption = reflector_absorption
reflector.fission = reflector_fission
reflector.scattering = reflector_scattering
reflector.chi = np.array([0, 0, 0, 0])

In [178]:
np.arange(3.0)

array([ 0.,  1.,  2.])

# Define the matrix for each group

## Fuel matrix (all groups)

In [143]:
def matrices(fuel, reflector, fuel_nodes, reflector_nodes, num_groups):
    n_fuel = fuel_nodes - 1
    n_reflector = reflector_nodes - 1
    nodes = n_fuel + n_reflector
    
    mats = np.zeros((num_groups, nodes, nodes))
    
    for g in range(num_groups):
        mat = np.eye(n_fuel + n_reflector)
        
        d_fuel = 1 / (3 * fuel.transport[g])
        dr_fuel = fuel.width / n_fuel
        
        d_ref = 1 / (3 * reflector.transport[g])
        dr_ref = reflector.width / n_reflector
        
        # Set most of the nodes
        for node in range(1, n_fuel + n_reflector - 1):
            if node < fuel_nodes:
                a = fuel.absorption[g]
                d = d_fuel
                dr = dr_fuel
            else:
                a = reflector.absorption[g]
                d = d_ref
                dr = dr_ref
                
            mat[node][node] = 2 * d / dr ** 2 + a
            mat[node][node + 1] = -d / dr ** 2 * (1 + 1 / (2 * node - 1))
            mat[node][node - 1] = -d / dr ** 2 * (1 - 1 / (2 * node - 1))
        
        # Boundary node
        mat[n_fuel][n_fuel - 1] = -d_fuel / dr_fuel + d_fuel / (2 * fuel.width)
        mat[n_fuel][n_fuel] = d_fuel / dr_fuel - d_fuel / (2 * fuel.width) +\
                                fuel.absorption[g] +\
                                d_ref / dr_ref + d_ref / fuel.width +\
                                reflector.absorption[g] * dr_ref / 2
        mat[n_fuel][n_fuel + 1] = -d_ref / dr_ref + -d_ref / (2 * fuel.width)
        
        # Center node (0)
        mat[0][0] = 0.5 * fuel.absorption[g] + d_fuel / dr_fuel ** 2
        mat[0][1] = -d_fuel / dr_fuel ** 2 - d_fuel / (dr ** 2)
        
        # Final (non-zero) node
        mat[-1][-2] = -d_ref / dr_ref ** 2 * (1 - 1 / (2 * (nodes + 1) - 1))
        mat[-1][-1] = 2 * d_ref / dr_ref ** 2 + reflector.absorption[g]
        
        mats[g,:,:] = mat
        
    print mats

In [144]:
matrices(fuel, reflector, 3, 4, 4)

[[[  2.514e-03  -1.317e-04   0.000e+00   0.000e+00   0.000e+00]
  [ -0.000e+00   5.027e-03  -8.104e-05   0.000e+00   0.000e+00]
  [  0.000e+00  -2.340e-03   6.239e-02  -3.676e-02   0.000e+00]
  [  0.000e+00   0.000e+00  -4.911e-04   1.738e-03  -7.366e-04]
  [  0.000e+00   0.000e+00   0.000e+00  -5.580e-04   1.738e-03]]

 [[  1.440e-03  -6.619e-05   0.000e+00   0.000e+00   0.000e+00]
  [ -0.000e+00   2.881e-03  -4.073e-05   0.000e+00   0.000e+00]
  [  0.000e+00  -1.176e-03   1.093e-01  -1.258e-02   0.000e+00]
  [  0.000e+00   0.000e+00  -1.681e-04   3.960e-03  -2.521e-04]
  [  0.000e+00   0.000e+00   0.000e+00  -1.910e-04   3.960e-03]]

 [[  1.528e-02  -3.848e-05   0.000e+00   0.000e+00   0.000e+00]
  [ -0.000e+00   3.055e-02  -2.368e-05   0.000e+00   0.000e+00]
  [  0.000e+00  -6.838e-04   4.522e-01  -1.333e-02   0.000e+00]
  [  0.000e+00   0.000e+00  -1.781e-04   1.626e-02  -2.671e-04]
  [  0.000e+00   0.000e+00   0.000e+00  -2.024e-04   1.626e-02]]

 [[  6.051e-02  -2.158e-05   0.000

In [159]:
def inscatter(material, from_group, to_group):
    return material.scattering[from_group][to_group]

In [200]:
def sources(fuel, reflector, fuel_nodes, reflector_nodes, flux, num_groups):
    n_fuel = fuel_nodes - 1
    n_reflector = reflector_nodes - 1
    
    nodes = n_fuel + n_reflector
    
    srcs = np.zeros((num_groups, nodes))
    
    for g in range(num_groups):
        src = np.ones(nodes)
        flx = flux[g,:]
        
        for i in range(nodes):
            src[i] = 0
            
            if i < fuel_nodes:
                scattering = fuel.scattering
                chi = fuel.chi
                nu = fuel.neutrons_per_fission
                fission = fuel.fission
                material = fuel
            else:
                scattering = reflector.scattering
                chi = reflector.chi
                nu = reflector.chi
                fission = reflector.fission
                material = reflector
            
            src[i] += flx[i] * chi[g] * fission[g]
            for j in range(num_groups):
                src[i] += inscatter(material, j, g) * flx[i]
            
        srcs[g,:] = src
    print srcs
    

In [202]:
num_groups = 4
fluxes = np.ones((num_groups, 6))
sources(fuel, reflector, 4, 4, flux, num_groups)

[[ 0.01   0.01   0.01   0.01   0.37   0.37 ]
 [ 0.083  0.083  0.083  0.083  1.024  1.024]
 [ 0.058  0.058  0.058  0.058  0.837  0.837]
 [ 0.065  0.065  0.065  0.065  2.298  2.298]]


# Start calculating flux for each group

In [180]:
def power(group, min_error=0.00001, max_iterations=1e5):
    flux = fluxes[group - 1]
    k_eff = 1.0
    flux_old = np.ones(nodes-1) * 100.
    k_eff_old = 1.0
    
    flux_error = 1.0
    k_eff_error = 1.0
    
    iterations = 0
    
    source = np.ones_like(flux) * nusig
    source[0] = source[0] * 0.5
    
    mat = matrix(nodes, width, absorption, transfer)
    
    while ((flux_error > min_error) or (k_eff_error > min_error)) and (iterations < max_iterations):
        # iterate generations
        iterations += 1
        
        flux = la.solve(mat, np.multiply(flux, source) / k_eff)
        
        # solve for eigenvalue
        k_eff = k_eff_old * la.norm(np.multiply(flux, source)) / la.norm(np.multiply(flux_old, source))
        
        flux = flux / la.norm(flux)
        
        flux_error = np.amax(abs((np.multiply(flux, source) - np.multiply(flux_old, source)) / np.multiply(flux, source)))
        k_eff_error = (abs((k_eff - k_eff_old) / k_eff))
        
        k_eff_old = k_eff
        flux_old = flux
        
        if (iterations % 10000 == 0):
            print flux_error
    
    return flux, k_eff, flux_error, k_eff_error, iterations

In [10]:
a = np.array(range(25)).reshape((5,5))
z = np.zeros(5)

In [11]:
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [12]:
z

array([ 0.,  0.,  0.,  0.,  0.])

In [23]:
for i in range(5):
    a = np.vstack((a, z))

In [24]:
a

array([[  0.,   1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.],
       [ 15.,  16.,  17.,  18.,  19.],
       [ 20.,  21.,  22.,  23.,  24.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.]])

In [30]:
a.shape[0]

10

In [40]:
np.hstack((a, np.zeros(a.shape[0]).reshape((a.shape[0], 1))))

array([[  0.,   1.,   2.,   3.,   4.,   0.],
       [  5.,   6.,   7.,   8.,   9.,   0.],
       [ 10.,  11.,  12.,  13.,  14.,   0.],
       [ 15.,  16.,  17.,  18.,  19.,   0.],
       [ 20.,  21.,  22.,  23.,  24.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])

In [39]:
np.zeros(a.shape[0])

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [37]:
[np.zeros(a.shape[0])]

[array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]

In [41]:
a

array([[  0.,   1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.],
       [ 15.,  16.,  17.,  18.,  19.],
       [ 20.,  21.,  22.,  23.,  24.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.]])

In [46]:
a = np.vsplit(a, 2)[0]

In [47]:
a

array([[  0.,   1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.],
       [ 15.,  16.,  17.,  18.,  19.],
       [ 20.,  21.,  22.,  23.,  24.]])

In [52]:
z = np.array(range(36)).reshape((6,6))

In [53]:
z

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35]])

In [54]:
np.zeros((a.shape[0], z.shape[1])).shape

(5, 6)

In [55]:
np.zeros((z.shape[0], a.shape[1])).shape

(6, 5)

In [57]:
t_left = np.vstack((a, np.zeros((z.shape[0], a.shape[1]))))
t_right = np.vstack((np.zeros((a.shape[0], z.shape[1])), z))
m = np.hstack((t_left, t_right))
m

array([[  0.,   1.,   2.,   3.,   4.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  5.,   6.,   7.,   8.,   9.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 10.,  11.,  12.,  13.,  14.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 15.,  16.,  17.,  18.,  19.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 20.,  21.,  22.,  23.,  24.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   1.,   2.,   3.,   4.,   5.],
       [  0.,   0.,   0.,   0.,   0.,   6.,   7.,   8.,   9.,  10.,  11.],
       [  0.,   0.,   0.,   0.,   0.,  12.,  13.,  14.,  15.,  16.,  17.],
       [  0.,   0.,   0.,   0.,   0.,  18.,  19.,  20.,  21.,  22.,  23.],
       [  0.,   0.,   0.,   0.,   0.,  24.,  25.,  26.,  27.,  28.,  29.],
       [  0.,   0.,   0.,   0.,   0.,  30.,  31.,  32.,  33.,  34.,  35.]])